In [355]:
import numpy as np
from scipy.sparse import random
import matplotlib.pyplot as plot

In [363]:
V = 2 # number of features
R = 20 # size of reservoir
LEARN_TIME = 10
READOUT_TIME = 50
LEARNING_RATE = 0.001
DENSITY = 0.25
NUM_ITERATIONS = 10

Wr = random(R, R, density=DENSITY).toarray()
# Win = random(V, R, density=DENSITY).toarray()
Win = np.random.random((V, R))
W_readout = np.random.random((R, 1))

X = np.array([[0, 0, 1, 1], [0, 1, 0, 1]])
h = np.random.random((R, 1))
y = np.array([0, 0, 0, 1]) 
delta = []

In [357]:
def advance_state(activation_function):
    global h, x, Win, Wr, delta
    h_new = np.matmul(Win.transpose(), x) + np.matmul(Wr.transpose(), h)
    h_new = activation_function(h_new)
    delta.append(np.sum(h_new - h))
    h = h_new

In [358]:
def relu(x):
    if x > 0:
	    return x
    else:
        return 0
    
def linear(x):
	return x

def sigmoid(x):
	return 1/(1 + np.exp(-x))

In [359]:
def forward(x):
	global Win, Wr, h
	predict = np.matmul(Win.transpose(), x) + np.matmul(Wr.transpose(), h)
	return np.matmul(W_readout.transpose(), predict)

In [364]:
for epoch in range(NUM_ITERATIONS):
	permutation = np.random.permutation(4)
	
	for datapoint in permutation:
		x = X[:, datapoint].reshape(2, 1)
		label = y[datapoint]
		
		for iteration in range(LEARN_TIME):
		    advance_state(sigmoid)

		# global delta
		# fig, ax = plot.subplots()
		# ax.plot(range(delta.__len__()), delta)
		# delta = []
		errors = []
		for iteration in range(READOUT_TIME):
			predict = forward(x)
			W_readout = W_readout - LEARNING_RATE * 2 * (predict - label)
			errors.append(np.square(predict - label).item(0))

		predict = forward(x)
		print(x[0], x[1], predict)

for datapoint in range(4):
	x = X[:, datapoint].reshape(2, 1)
	print(x[0], x[1], forward(x))

[1] [0] [[0.06804514]]
[0] [1] [[0.00107384]]
[0] [0] [[-0.00132393]]
[1] [1] [[0.99942187]]
[0] [0] [[0.00602126]]
[0] [1] [[0.00039617]]
[1] [1] [[0.99929568]]
[1] [0] [[0.00132925]]
[0] [0] [[0.00197696]]
[1] [1] [[0.99942447]]
[0] [1] [[0.00211032]]
[1] [0] [[-0.00107779]]
[0] [1] [[0.00094495]]
[1] [1] [[0.99929601]]
[1] [0] [[0.00132925]]
[0] [0] [[0.00197696]]
[0] [1] [[0.00038663]]
[0] [0] [[-0.00132829]]
[1] [0] [[-0.00064924]]
[1] [1] [[0.99961106]]
[0] [1] [[0.0021106]]
[1] [1] [[0.99929672]]
[0] [0] [[0.00602059]]
[1] [0] [[-0.00062945]]
[0] [0] [[0.00196424]]
[0] [1] [[0.0003866]]
[1] [1] [[0.99929567]]
[1] [0] [[0.00132925]]
[1] [0] [[2.83246139e-06]]
[0] [0] [[0.00196834]]
[1] [1] [[0.99942446]]
[0] [1] [[0.00211032]]
[0] [1] [[3.84280805e-06]]
[0] [0] [[-0.00133071]]
[1] [0] [[-0.00064925]]
[1] [1] [[0.99961106]]
[1] [0] [[0.0013298]]
[0] [1] [[0.00094944]]
[1] [1] [[0.99929602]]
[0] [0] [[0.00602058]]
[0] [0] [[0.00602058]]
[0] [1] [[0.13040849]]
[1] [0] [[-0.35611031]

In [361]:
def thresh(x):
	if x >= .5:
		return 1
	return 0

In [362]:
print(thresh(forward(X[:, 0].reshape(2,1))))
print(thresh(forward(X[:, 1].reshape(2,1))))
print(thresh(forward(X[:, 2].reshape(2,1))))
print(thresh(forward(X[:, 3].reshape(2,1))))

0
0
0
0
